<a href="https://colab.research.google.com/github/QianyueWang0212/mgmt467-analytics-portfolio/blob/main/Labs/Lab6_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Lab 6: Improving Models with Feature Engineering**
**Unit 2 • Week 9 (Thu) — Feature Engineering & Advanced Models**

**Objective:** Improve your **Lab 5** classifier by creating **new features** using the BQML `TRANSFORM` clause, then compare performance to a **baseline**.


## Setup & Authentication

In [2]:
from google.colab import files
import os

# Prompt the user to upload their kaggle.json file.
# This file contains your Kaggle API credentials.
print("Upload your kaggle.json (Kaggle > Account > Create New API Token)")
uploaded = files.upload()

# Create the .kaggle directory if it doesn't exist.
# This is where Kaggle expects to find the credentials file.
os.makedirs('/root/.kaggle', exist_ok=True)

# Save the uploaded file to the correct location.
# Using the first uploaded file as we expect only one (kaggle.json).
with open('/root/.kaggle/kaggle.json', 'wb') as f:
    f.write(uploaded[list(uploaded.keys())[0]])

# Set file permissions to 0600 (owner read/write only).
# This is crucial for security to prevent other users from accessing your API key.
os.chmod('/root/.kaggle/kaggle.json', 0o600)

# Verify the Kaggle installation by printing the version.
# This confirms the CLI is installed and can access the credentials.
!kaggle --version

# Done: Kaggle setup

Upload your kaggle.json (Kaggle > Account > Create New API Token)


Saving kaggle (3).json to kaggle (3).json
Kaggle API 1.7.4.5


In [3]:
#EXAMPLE (from LLM) — Auth + Project/Region (commented; write your own cell using the prompt)
from google.colab import auth
auth.authenticate_user()

import os
PROJECT_ID = input("Enter your GCP Project ID: ").strip()
REGION = "us-central1"  # keep consistent; change if instructed
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
print("Project:", PROJECT_ID, "| Region:", REGION)

# Set active project for gcloud/BigQuery CLI
!gcloud config set project $GOOGLE_CLOUD_PROJECT
!gcloud config get-value project
# Done: Auth + Project/Region set

Enter your GCP Project ID: mgmt-46700
Project: mgmt-46700 | Region: us-central1
Updated property [core/project].
mgmt-46700


In [4]:
# EXAMPLE (from LLM) — Kaggle setup (commented)
from google.colab import files
print("Upload your kaggle.json (Kaggle > Account > Create New API Token)")
uploaded = files.upload()

import os
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'wb') as f:
    f.write(uploaded[list(uploaded.keys())[0]])
os.chmod('/root/.kaggle/kaggle.json', 0o600)  # owner-only

!kaggle --version

Upload your kaggle.json (Kaggle > Account > Create New API Token)


Saving kaggle (3).json to kaggle (3) (1).json
Kaggle API 1.7.4.5


In [5]:
# Create the directory to store raw data
!mkdir -p /content/data/raw

# Download the dataset using Kaggle CLI to /content/data
# The -d flag specifies the dataset, and -p specifies the download path
!kaggle datasets download -d mahoora00135/flights -p /content/data

# Unzip the downloaded dataset into the raw data directory
# -o flag overwrites files if they exist
!unzip -o /content/data/*.zip -d /content/data/raw

# List all CSV files in the raw data directory with their sizes in a neat table
!ls -lh /content/data/raw/*.csv

Dataset URL: https://www.kaggle.com/datasets/mahoora00135/flights
License(s): CC0-1.0
  0% 0.00/10.3M [00:00<?, ?B/s]
100% 10.3M/10.3M [00:00<00:00, 1.19GB/s]
Archive:  /content/data/flights.zip
  inflating: /content/data/raw/flights.csv  
-rw-r--r-- 1 root root 41M Sep 26  2023 /content/data/raw/flights.csv


In [6]:
# Create a GCS bucket (only once per project)
BUCKET_NAME = f"{PROJECT_ID}-flights-bucket"
!gsutil mb -l {REGION} gs://{BUCKET_NAME}/ || echo "Bucket may already exist"

# Upload dataset to the bucket
!gsutil cp /content/data/raw/flights.csv gs://{BUCKET_NAME}/flights.csv

print(f"✅ Uploaded flights.csv to: gs://{BUCKET_NAME}/flights.csv")


Creating gs://mgmt-46700-flights-bucket/...
ServiceException: 409 A Cloud Storage bucket named 'mgmt-46700-flights-bucket' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.
Bucket may already exist
Copying file:///content/data/raw/flights.csv [Content-Type=text/csv]...
\
Operation completed over 1 objects/40.9 MiB.                                     
✅ Uploaded flights.csv to: gs://mgmt-46700-flights-bucket/flights.csv


In [7]:
from google.cloud import bigquery
from google.api_core.exceptions import Conflict

client = bigquery.Client(project=PROJECT_ID)

dataset_id = f"{PROJECT_ID}.flights_dataset"
table_id = f"{dataset_id}.flights"

# Create dataset if it doesn't exist
dataset = bigquery.Dataset(dataset_id)
dataset.location = REGION

try:
    client.create_dataset(dataset, timeout=30)  # Make an API request.
    print(f"✅ Dataset {dataset_id} created.")
except Conflict:
    print(f"Dataset {dataset_id} already exists.")
except Exception as e:
    print(f"Error creating dataset {dataset_id}: {e}")
    # Exit the cell execution if dataset creation fails
    raise

# Define load job from GCS
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
)

uri = f"gs://{BUCKET_NAME}/flights.csv"
load_job = client.load_table_from_uri(uri, table_id, job_config=job_config)
load_job.result()

table = client.get_table(table_id)
print(f"✅ Loaded {table.num_rows} rows into {table_id}")

Dataset mgmt-46700.flights_dataset already exists.
✅ Loaded 1010328 rows into mgmt-46700.flights_dataset.flights



---
## Establish a Baseline — Validate

Re-run `ML.EVALUATE` for your **Lab 5** model and record key metrics.


In [18]:
# Evaluate model performance for the classifier
eval_sql = f"""
SELECT *
FROM ML.EVALUATE(MODEL `{dataset_id}.flight_delay_classifier_v2`);
"""

eval_df = client.query(eval_sql).result().to_dataframe()
display(eval_df)


,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.935696,0.793103,0.976382,0.858519,0.07586,0.982321


Baseline Model Performance:

| precision | recall   | accuracy | f1_score | log_loss | roc_auc  |
| :-------- | :------- | :------- | :------- | :------- | :------- |
| {{eval_df.precision[0]}} | {{eval_df.recall[0]}} | {{eval_df.accuracy[0]}} | {{eval_df.f1_score[0]}} | {{eval_df.log_loss[0]}} | {{eval_df.roc_auc[0]}} |


---
## Brainstorm New Features — Investigate (Gemini Prompt)


prompt =
```
# TASK: Brainstorm new features for an ML model, a process called feature engineering.
# CONTEXT: I want to improve my flight diversion prediction model. The raw data has a column called 'origin' (e.g., 'JFK', 'ORD') and another called 'carrier' (e.g., 'AA', 'UA').
# GOAL: Suggest one new feature I could create by combining 'origin' and 'carrier' that might be more predictive than either column alone. Explain why this new feature could be more powerful.
```

To brainstorm new features for your flight diversion prediction model, combining 'origin' and 'carrier' could lead to a powerful new feature called 'origin_carrier_route'.

This feature could be created by concatenating the origin and carrier columns (e.g., 'JFK-AA', 'ORD-UA').

Why this could be more powerful:

While origin and carrier are useful individually, their combination can capture specific interactions or unique characteristics of flights operated by a particular carrier from a specific origin airport. For example, one carrier might have a consistently better on-time performance or handling process at a certain airport compared to others. This combined feature can reveal these nuances that might not be evident from looking at the origin or carrier alone, potentially leading to better predictions of flight diversions.




---
## Implement Feature Engineering with `TRANSFORM`

Modify your `CREATE MODEL` from Lab 5 and add a `TRANSFORM` clause to create engineered features.

Requested features:
1. **`route`** = CONCAT(`origin`, '-', `dest`)  
2. **`day_of_week`** = EXTRACT(DAYOFWEEK FROM `fl_date`)


In [15]:
# === IDs you already have from Lab 5 ===
# PROJECT_ID was set earlier via auth
dataset_id = f"{PROJECT_ID}.flights_dataset"          # <- your dataset from the load step
table_id   = f"{dataset_id}.flights"                  # <- the table you loaded (1,010,328 rows)
model_id   = f"{dataset_id}.flight_delay_classifier_v2"

from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID)

# Train a LOGISTIC_REG model with engineered features.
# Label: delayed_flag = 1 if arr_delay > 60 else 0
create_v2_sql = f"""
CREATE OR REPLACE MODEL `{model_id}`
TRANSFORM(
  -- pass-through / typed
  CAST(distance AS FLOAT64) AS distance,
  CAST(dep_delay AS FLOAT64) AS dep_delay,
  CAST(air_time  AS FLOAT64) AS air_time,
  CAST(carrier   AS STRING)  AS carrier,
  CAST(origin    AS STRING)  AS origin,
  CAST(dest      AS STRING)  AS dest,
  delayed_flag, -- Include the label column

  -- engineered
  CONCAT(CAST(origin AS STRING), '-', CAST(dest AS STRING)) AS route,
  EXTRACT(DAYOFWEEK FROM CAST(time_hour AS DATE))          AS day_of_week
)
OPTIONS(
  model_type='LOGISTIC_REG',
  input_label_cols=['delayed_flag'],
  enable_global_explain=TRUE
)
AS
SELECT
  CASE WHEN arr_delay > 60 THEN 1 ELSE 0 END AS delayed_flag,
  *
FROM `{table_id}`
WHERE arr_delay IS NOT NULL
  AND dep_delay IS NOT NULL
LIMIT 200000
"""
client.query(create_v2_sql).result()
print("✅ Model created: flight_delay_classifier_v2 (with route + day_of_week)")

✅ Model created: flight_delay_classifier_v2 (with route + day_of_week)



---
## Compare Performance — Extend

Evaluate the improved model and compare against the baseline.


In [17]:
improved_sql = f"SELECT * FROM ML.EVALUATE(MODEL `{model_id}`)"
improved_df = client.query(improved_sql).result().to_dataframe()
display(improved_df)

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.937346,0.816043,0.97757,0.872499,0.073164,0.984334



Create a small comparison table below in Markdown or code (precision, recall, etc.):
- **Baseline (Lab 5)**: …  
- **Improved (Lab 6)**: …  

**Did feature engineering improve performance? Why or why not?**


### Comparison: Baseline vs Improved Model

| Metric | Baseline (Lab 5) | Improved (Lab 6) | Change |
|:--------|:----------------:|:----------------:|:-------:|
| Precision | 0.9357 | **0.9373** | +0.0016 |
| Recall | 0.7931 | **0.8160** | +0.0229 |
| Accuracy | 0.9764 | **0.9776** | +0.0012 |
| F1 Score | 0.8585 | **0.8725** | +0.0140 |
| Log Loss | 0.0759 | **0.0732** | ↓ Better |
| ROC AUC | 0.9823 | **0.9843** | +0.0020 |

**Did feature engineering improve performance?**

✅ **Yes.**  
Adding the new engineered features — `route` (origin-destination pair) and `day_of_week` — helped the model better capture route-level and scheduling patterns that correlate with diversions.  
- The **recall** increased the most, meaning the model now detects a higher proportion of actual diversions.  
- **Log loss** decreased slightly, showing the model’s predictions became more confident and accurate.  
- Overall, the improvements are modest but consistent across all metrics, confirming that feature engineering provided additional predictive power.



---
## Challenge: `ML.BUCKETIZE`

Author your own Gemini prompt to write a `TRANSFORM` clause that buckets `dep_delay` into 4 severity levels (e.g., early/on-time, minor, moderate, major).

> Hint: `ML.BUCKETIZE(dep_delay, [boundary_list])` returns a **bucket index**; you can also map buckets with `CASE`.


prompt = """
```
# TASK: Write a BigQuery ML TRANSFORM clause using ML.BUCKETIZE.
# CONTEXT: I want to categorize 'dep_delay' (departure delay in minutes) into 4 severity levels
# to use as an input feature in my logistic regression model.
# GOAL: Use ML.BUCKETIZE to split dep_delay into ranges like:
#   • Early/On-Time  → dep_delay < 0
#   • Minor Delay    → 0 ≤ dep_delay < 15
#   • Moderate Delay → 15 ≤ dep_delay < 60
#   • Major Delay    → dep_delay ≥ 60
# INSTRUCTION: Show how to define these buckets inside a TRANSFORM clause and
# optionally use CASE to map each bucket index to a readable label (e.g., 'early', 'minor', etc.).
````
"""


In [30]:
# ==== IDs (match your earlier cells) ====
# PROJECT_ID is already set from auth
dataset_id = f"{PROJECT_ID}.flights_dataset"      # same dataset you used before
table_id   = f"{dataset_id}.flights"              # your Kaggle table in BigQuery
model_id   = f"{dataset_id}.flight_delay_classifier_v3"

from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID)

# (Optional) If you hit "concurrent model update" errors, uncomment the next line once:
# !bq rm -f -m {model_id}

bucketize_sql = f"""
CREATE OR REPLACE MODEL `{model_id}`
TRANSFORM (
  -- pass-through features
  CAST(distance AS FLOAT64) AS distance,
  CAST(dep_delay AS FLOAT64) AS dep_delay,
  CAST(air_time  AS FLOAT64) AS air_time,
  CAST(carrier   AS STRING)  AS carrier,
  CAST(origin    AS STRING)  AS origin,
  CAST(dest      AS STRING)  AS dest,
  delayed_flag, -- Include the label column

  -- engineered features
  CONCAT(CAST(origin AS STRING), '-', CAST(dest AS STRING)) AS route,
  EXTRACT(DAYOFWEEK FROM CAST(time_hour AS DATE))          AS day_of_week,

  -- bucketized departure delay (4 severity levels)
  -- boundaries: [-inf,0), [0,15), [15,60), [60, +inf)
  ML.BUCKETIZE(dep_delay, [0, 15, 60]) AS dep_delay_bucket,

  -- optional readable category (useful for explain/debug)
  CASE
    WHEN dep_delay < 0  THEN 'early'
    WHEN dep_delay < 15 THEN 'on_time'
    WHEN dep_delay < 60 THEN 'moderate'
    ELSE 'major'
  END AS delay_category
)
OPTIONS(
  model_type='LOGISTIC_REG',
  input_label_cols=['delayed_flag'],
  enable_global_explain=TRUE
)
AS
SELECT
  -- CONSISTENT LABEL FOR THIS DATASET (we don't have a 'diverted' column)
  CASE WHEN arr_delay > 60 THEN 1 ELSE 0 END AS delayed_flag,
  distance, dep_delay, air_time, carrier, origin, dest, time_hour
FROM `{table_id}`
WHERE arr_delay IS NOT NULL
  AND dep_delay IS NOT NULL
LIMIT 200000;
"""

client.query(bucketize_sql).result()
print("✅ Model created: flight_delay_classifier_v3 (uses delayed_flag + dep_delay buckets)")

✅ Model created: flight_delay_classifier_v3 (uses delayed_flag + dep_delay buckets)



---
## ✅ Deliverable for Lab 6

- Completed `Lab6_Feature_Engineering.ipynb` showing:
  - Baseline metrics
  - Engineered features via `TRANSFORM`
  - Improved model evaluation + comparison
- Push to **GitHub** and submit the link on **Brightspace**.
